In [32]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [33]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [34]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [35]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [36]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d', 
                                                                    'price_change_percentage_14d', 'price_change_percentage_30d', 
                                                                    'price_change_percentage_60d', 'price_change_percentage_200d', 
                                                                    'price_change_percentage_1y']])

In [37]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(market_data_scaled, columns = ['price_change_percentage_24h', 
                                                                    'price_change_percentage_7d', 
                                                                    'price_change_percentage_14d', 
                                                                    'price_change_percentage_30d', 
                                                                    'price_change_percentage_60d', 
                                                                    'price_change_percentage_200d', 
                                                                    'price_change_percentage_1y'])

# Copy the crypto names from the original data
df_market_data_scaled['crypto_names'] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index('crypto_names')

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
crypto_names,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [38]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [39]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k: 
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters = i, random_state = 0)
    # 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(df_market_data_scaled)
    # 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)

In [40]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k, 'inertia': inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [41]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve = df_elbow.hvplot.line(x = 'k', y = 'inertia', title = 'Elbow Curve', xticks = k)
elbow_curve

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [42]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4)

In [43]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4)

In [44]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 2 0 1 1 3
 1 1 1 1]


In [45]:
# Create a copy of the DataFrame
df_market_scaled_predictions = df_market_data_scaled.copy()

In [46]:
# Add a new column to the DataFrame with the predicted clusters
df_market_scaled_predictions['CryptoCluster'] = crypto_clusters

# Display sample data
df_market_scaled_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,CryptoCluster
crypto_names,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [47]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_clusters = df_market_scaled_predictions.hvplot.scatter(
    x = 'price_change_percentage_24h', y = 'price_change_percentage_7d', 
    color = 'CryptoCluster', hover_cols = ['crypto_names'], 
    title = 'Crypto Clusters with K-means Using the Original Data'
)
crypto_clusters

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,CryptoCluster,crypto_names)

---

### Optimize Clusters with Principal Component Analysis.

In [48]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)

In [49]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
market_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [50]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 89.5% of the total variance is condensed into the three principal components.

In [51]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(market_pca, columns = ['PCA1', 'PCA2', 'PCA3'])

# Copy the crypto names from the original data
df_market_data_pca['crypto_names'] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index('crypto_names')

# Display sample data
df_market_data_pca.head()

,PCA1,PCA2,PCA3
crypto_names,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [52]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [53]:
# Create an empy list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for i in k_values: 
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters = i, random_state = 0)
    # 2. Fit the model to the data using `df_market_data_pca`
    k_model.fit(df_market_data_pca)
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(k_model.inertia_)

In [54]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {'k': k_values, 'inertia': inertia_values}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [55]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve_pca = df_elbow_pca.hvplot.line(x = 'k', y = 'inertia', title = 'Elbow Curve using PCA Data', xticks = k)
elbow_curve_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best k value found using the PCA data seems to be the same as the best k value found using the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [56]:
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters = 4)

In [57]:
# Fit the K-Means model using the PCA data
model_pca.fit(df_market_data_pca)

KMeans(n_clusters=4)

In [58]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_clusters_pca = model_pca.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(crypto_clusters_pca)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 3 1 0 0 2
 0 0 0 0]


In [59]:
# Create a copy of the DataFrame with the PCA data
df_market_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_pca_predictions['CryptoCluster'] = crypto_clusters_pca

# Display sample data
df_market_pca_predictions.head()

,PCA1,PCA2,PCA3,CryptoCluster
crypto_names,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,1


In [60]:
# Create a scatter plot using hvPlot by setting 
# `x="PCA1"` and `y="PCA2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_pca_clusters = df_market_pca_predictions.hvplot.scatter(x = 'PCA1', y = 'PCA2',
                                                               color = 'CryptoCluster', hover_cols = ['crypto_names'], 
                                                               title = 'Crypto Clusters with K-means Using PCA Data' 
                                                              )
crypto_pca_clusters

:Scatter   [PCA1]   (PCA2,CryptoCluster,crypto_names)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [61]:
# Composite plot to contrast the Elbow curves
elbow_curve + elbow_curve_pca

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [62]:
# Composite plot to contrast the clusters
crypto_clusters + crypto_pca_clusters

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,CryptoCluster,crypto_names)
   .Scatter.II :Scatter   [PCA1]   (PCA2,CryptoCluster,crypto_names)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** With fewer features used to cluster data using K-means, it seems to reduce the amount of noise in the clusters and make the grouping of the data more clear and interpretable.